# Traitement de données graphes avec Spark GraphX

Dans ce notebook, nous allons utiliser Spark GraphX pour récupérer des information sur le graphe suivant, représentant un réseau social fictif : 

![Graphe](../images/graph.png)

In [ ]:
// Notebook startup
print("Hello, World!")